In [1]:
import string

In [2]:
contents = unicode(open('data/friends_transcripts.txt').read(), errors='ignore')
contents

u'======== 01-01\nWritten by: Marta Kauffman & David Crane\nMonica: There\'s nothing to tell! He\'s just some guy I work with!\nJoey: C\'mon, you\'re going out with the guy! There\'s gotta be something wrong with him!\nChandler: All right Joey, be nice. So does he have a hump? A hump and a hairpiece?\nPhoebe: Wait, does he eat chalk?\nPhoebe: Just, \'cause, I don\'t want her to go through what I went through with Carl- oh!\nMonica: Okay, everybody relax. This is not even a date. It\'s just two people going out to dinner and- not having sex.\nChandler: Sounds like a date to me.\nChandler: Alright, so I\'m back in high school, I\'m standing in the middle of the cafeteria, and I realize I am totally naked.\nAll: Oh, yeah. Had that dream.\nChandler: Then I look down, and I realize there\'s a phone... there.\nJoey: Instead of...?\nChandler: That\'s right.\nJoey: Never had that dream.\nPhoebe: No.\nChandler: All of a sudden, the phone starts to ring. Now I don\'t know what to do, everybody s

First element of ``re.split`` is empty string, so skip it:

In [3]:
import re

splitted = re.split('======== (\d\d)-(\d\d)\n', contents)[1:]
print(len(splitted))

666


In [4]:
from itertools import izip

def tripletswise(t):
    it = iter(t)
    return izip(it,it,it)

def clean(txt):
    for i in string.punctuation:
        txt = txt.replace(i, '')
    return txt

matched_data = [(int(x), int(y), clean(z)) for x, y, z in tripletswise(splitted)]

The first element is **season**,

The second is **episode**,

And the third – actual **transcript**:

In [5]:
documents = [d[2] for d in matched_data]

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

stemmed_doc = []
for doc in documents:
    tokens = nltk.word_tokenize(doc)
    filtered_tokens = [t for t in tokens if not t in stopwords.words('english')]
    
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(t) for t in filtered_tokens]
    
    stemmed_doc.append(stemmed)


In [11]:
from collections import Counter
doc_frequencies = Counter()
for doc in stemmed_doc:
    for w in set(doc):
        doc_frequencies[w] += 1

In [16]:
import math

def tfidf(word, freq, doc_freq):
    tf = freq
    idf = math.log(len(stemmed_doc) / (doc_frequencies[word] + 1))
    return tf * idf

for i in range(len(stemmed_doc)):
    counts = Counter(stemmed_doc[i])
    tfidf_weighted = sorted(
        counts.most_common(),
        key=lambda x: tfidf(x[0], x[1], doc_frequencies),
        reverse=True
    )
    
    print("Season {} episode {}: {}".format(matched_data[i][0], matched_data[i][1], tfidf_weighted[:5]))

Season 1 episode 1: [(u'Paul', 38), (u'Franni', 6), (u'Wine', 5), (u'cut', 15), (u'Goodnight', 4)]
Season 1 episode 2: [(u'Barri', 20), (u'Carol', 29), (u'Susan', 14), (u'Robbi', 6), (u'ew', 7)]
Season 1 episode 3: [(u'Alan', 23), (u'Lizzi', 12), (u'Paula', 6), (u'smoke', 10), (u'Chi', 4)]
Season 1 episode 4: [(u'Pizza', 8), (u'Joann', 6), (u'Kiki', 5), (u'Receptionist', 6), (u'omnipot', 4)]
Season 1 episode 5: [(u'Angela', 18), (u'Bob', 15), (u'Janic', 16), (u'laundri', 11), (u'sud', 5)]
Season 1 episode 6: [(u'Aurora', 31), (u'Ree', 9), (u'Ethan', 7), (u'Rick', 6), (u'Pacino', 5)]
Season 1 episode 7: [(u'Paolo', 21), (u'Jill', 21), (u'OK', 19), (u'vestibul', 5), (u'gum', 7)]
Season 1 episode 8: [(u'Shelley', 15), (u'Lowel', 11), (u'Geller', 39), (u'Lillian', 9), (u'Nana', 9)]
Season 1 episode 9: [(u'Thanksgiv', 17), (u'shoop', 9), (u'Susan', 13), (u'key', 16), (u'Ok', 19)]
Season 1 episode 10: [(u'Max', 19), (u'David', 32), (u'Minsk', 12), (u'Bobbi', 9), (u'Fun', 9)]
Season 1 episode